In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

In [3]:
df = pd.read_csv('alpha24_step15Stud.csv')
df.head()

,czas(min),alpha,temp,Ts
0,0,1,"6,50",2
1,15,0,"6,35","2,18"
2,30,0,"6,20","2,19"
3,45,1,"6,05","1,68"
4,60,1,"5,90","1,5"


In [4]:
df['temp'] = df['temp'].apply(lambda x: x.replace(',','.')).astype('float64')
df['Ts'] = df['Ts'].apply(lambda x: x.replace(',','.')).astype('float64')

In [5]:
df.head()

,czas(min),alpha,temp,Ts
0,0,1,6.50,2.00
1,15,0,6.35,2.18
2,30,0,6.20,2.19
3,45,1,6.05,1.68
4,60,1,5.90,1.50


In [6]:
df.rename(columns={"czas(min)": "time", "alpha": "compressor", "temp": 'ambient_temp', "Ts": 'refrigerator_temp'}, inplace=True)
df.set_index('time', inplace=True)
df.head()

,compressor,ambient_temp,refrigerator_temp
time,,,
0,1,6.50,2.00
15,0,6.35,2.18
30,0,6.20,2.19
45,1,6.05,1.68
60,1,5.90,1.50


In [7]:
size = 6
delay = 3
a = df.iloc[0:size,:].to_numpy().reshape(delay,size*3//delay)
a

array([[1.  , 6.5 , 2.  , 0.  , 6.35, 2.18],
       [0.  , 6.2 , 2.19, 1.  , 6.05, 1.68],
       [1.  , 5.9 , 1.5 , 0.  , 5.74, 1.64]])

In [8]:
for predictor in range(a.shape[1] - 1):
    sorted_predictor = a[:,predictor].argsort()
    b = a[sorted_predictor]
    print(predictor, b, "\n")


0 [[0.   6.2  2.19 1.   6.05 1.68]
 [1.   6.5  2.   0.   6.35 2.18]
 [1.   5.9  1.5  0.   5.74 1.64]] 

1 [[1.   5.9  1.5  0.   5.74 1.64]
 [0.   6.2  2.19 1.   6.05 1.68]
 [1.   6.5  2.   0.   6.35 2.18]] 

2 [[1.   5.9  1.5  0.   5.74 1.64]
 [1.   6.5  2.   0.   6.35 2.18]
 [0.   6.2  2.19 1.   6.05 1.68]] 

3 [[1.   6.5  2.   0.   6.35 2.18]
 [1.   5.9  1.5  0.   5.74 1.64]
 [0.   6.2  2.19 1.   6.05 1.68]] 

4 [[1.   5.9  1.5  0.   5.74 1.64]
 [0.   6.2  2.19 1.   6.05 1.68]
 [1.   6.5  2.   0.   6.35 2.18]] 



In [9]:
SSR = lambda a: ((a.mean() - a)**2).sum()

for i in range(1,b.shape[0]):
    ssr_1 = SSR(b[:i, -1])
    ssr_2 = SSR(b[i:, -1])
    print(f"array: {b[:i, -1]} ssr = {ssr_1} \n\
             array {b[i:, -1]} ssr = {ssr_2} \n\
             SSR={ssr_1+ssr_2}")

array: [1.64] ssr = 0.0 
             array [1.68 2.18] ssr = 0.1250000000000001 
             SSR=0.1250000000000001
array: [1.64 1.68] ssr = 0.0008000000000000014 
             array [2.18] ssr = 0.0 
             SSR=0.0008000000000000014


In [31]:
print(a)
predictor = 4
value = 6.2
print(a[np.argwhere(a[:,predictor] <= value).flatten()])
print(a[np.argwhere(a[:,predictor] > value).flatten()])

[[1.   6.5  2.   0.   6.35 2.18]
 [0.   6.2  2.19 1.   6.05 1.68]
 [1.   5.9  1.5  0.   5.74 1.64]]
[[0.   6.2  2.19 1.   6.05 1.68]
 [1.   5.9  1.5  0.   5.74 1.64]]
[[1.   6.5  2.   0.   6.35 2.18]]


In [17]:
left = np.split(a, np.argwhere(a[:,4] <= 1.68)[0][1:])
a

IndexError: index 0 is out of bounds for axis 0 with size 0